Copyright (c) Microsoft Corporation.
Licensed under the MIT License.

# Abstractive Summarization using BertSumAbs on CNN/DailyMails Dataset

## Summary

This notebook demonstrates how to fine tune BERT for abstractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

### Abstractive Summarization
Abstractive summarization is the task of taking an input text and summarizing its content in a shorter output text. In contrast to extractive summarization, abstractive summarization doesn't take sentences directly from the input text, instead, rephrases the input text.

### BertSumAbs

BertSumAbs refers to an BERT-based abstractive summarization algorithm  in [Text Summarization with Pretrained Encoders](https://arxiv.org/abs/1908.08345) with [published examples](https://github.com/nlpyang/PreSumm). It uses the pretrained BERT model as encoder and finetune both encoder and decoder on a specific labeled summarization dataset like [CNN/DM dataset](https://github.com/harvardnlp/sent-summary). 

The figure below shows the comparison of architecture of the original BERT model (left) and BERTSUM (right), which BertSumAbs is built upon. For BERTSUM, a input document is split into sentences, and [CLS] and [SEP] tokens are inserted before and after each sentence. This resulting sequence is followed by the summation of three kinds of embeddings for each token before feeding into the transformer layers. The positional embedding used in BertSumAbs enables input length of more than 512, which is the  maximum input length for BERT model. 

It should be noted that the architecture only shows the encoder part. For decoder, BertSumAbs also uses a transformer with multiple layers and random initialization. As pretrained weights are used in the encoder, there is a mismatch in encoder and decoder which may result in unstable finetuning. Therefore, in fine tuning, BertSumAbs uses seperate optimizers for encoder and decoder, each uses its own scheduling. In text generation, techniques like trigram blocking and beam search can be used to improve model accuracy.
<img src="https://nlpbp.blob.core.windows.net/images/BertForSummarization.PNG">


## Before you start

It's recommended to run this notebook on GPU machines as it's very computationally intensive. Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of steps. If QUICK_RUN = False, the notebook takes about 5 hours to run on a VM with 4 16GB NVIDIA V100 GPUs. Finetuning costs around 1.5 hours and inferecing costs around 3.5 hour.  Better performance can be achieved by increasing the MAX_STEPS.

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [abstractive_summarization_bertsum_cnndm_distributed_train.py](./abstractive_summarization_bertsum_cnndm_distributed_train.py) shows an example of how to use DDP.

* **Mixed Precision Training**:
Please note that by default this notebook doesn't use mixed precision training. Faster speed and larger batch size can be achieved when you set FP16 to True. Refer to  https://nvidia.github.io/apex and https://github.com/nvidia/apex) for details to use mixed precision training. Check the GPU model on your machine to see if it allows mixed precision training. Please also note that mixed precision inferencing is also enabled in the prediciton utility function. When you use mixed precision training and/or inferencing, the model performance can be slightly worse than the full precision mode.

In [50]:
!pip install transformers pyrouge rouge indic-nlp-library>=0.6 TensorboardX==1.8 torchtext>=0.4.0 googledrivedownloader>=0.4
!pip install jsonlines>=1.2.0 nteract-scrapbook>=0.2.1

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.
ERROR: jupyterlab 1.2.6 has requirement jupyterlab_server~=1.0.0, but you'll have jupyterlab-server 1.1.0 which is incompatible.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
QUICK_RUN = True

In [53]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

print(os.getcwd())
nlp_path = os.path.abspath("../../../")
print(nlp_path)
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.models.transformers.abstractive_summarization_bertsum import (
    BertSumAbs,
    BertSumAbsProcessor,
)

from utils_nlp.dataset.cnndm import CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import pprint
import scrapbook as sb

/home/ec2-user/SageMaker/nlp-recipes/examples/text_summarization/abstractive
/home/ec2-user/SageMaker/nlp-recipes


## Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples. The length of the news articles is 781 tokens on average and the summaries are of 3.75 sentences and 56 tokens on average.

The significant part of data preprocessing only involve splitting the input document into sentences.

In [54]:
# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 100
if not QUICK_RUN:
    TOP_N = -1

In [55]:
train_dataset, test_dataset = CNNDMSummarizationDataset(
    top_n=TOP_N, local_cache_path=DATA_PATH, prepare_extractive=False
)

100%|██████████| 489k/489k [00:09<00:00, 49.1kKB/s] 


In [56]:
len(train_dataset)

100

In [57]:
len(test_dataset)

100

In [58]:
type(train_dataset)

utils_nlp.models.transformers.datasets.SummarizationDataset

## Model Finetuning

In [59]:
# notebook parameters
# the cache path
CACHE_PATH = TemporaryDirectory().name

# model parameters
MODEL_NAME = "bert-base-uncased"
MAX_POS = 768
MAX_SOURCE_SEQ_LENGTH = 640
MAX_TARGET_SEQ_LENGTH = 140

# mixed precision setting. To enable mixed precision training, follow instructions in SETUP.md.
FP16 = False
if FP16:
    FP16_OPT_LEVEL = "O2"

# fine-tuning parameters
# batch size, unit is the number of tokens
BATCH_SIZE_PER_GPU = 1


# GPU used for training
NUM_GPUS = torch.cuda.device_count()
if NUM_GPUS > 0:
    BATCH_SIZE = NUM_GPUS * BATCH_SIZE_PER_GPU
else:
    BATCH_SIZE = 1


# Learning rate
LEARNING_RATE_BERT = 5e-4 / 2.0
LEARNING_RATE_DEC = 0.05 / 2.0


# How often the statistics reports show up in training, unit is step.
REPORT_EVERY = 10
SAVE_EVERY = 500

# total number of steps for training
MAX_STEPS = 1e3

if not QUICK_RUN:
    MAX_STEPS = 5e3

WARMUP_STEPS_BERT = 2000
WARMUP_STEPS_DEC = 1000

In [60]:
# processor which contains the colloate function to load the preprocessed data
processor = BertSumAbsProcessor(cache_dir=CACHE_PATH, max_src_len=MAX_SOURCE_SEQ_LENGTH, max_tgt_len=MAX_TARGET_SEQ_LENGTH)
# summarizer
summarizer = BertSumAbs(
    processor, cache_dir=CACHE_PATH, max_pos_length=MAX_POS
)

In [61]:
BATCH_SIZE_PER_GPU*NUM_GPUS

1

In [ ]:
summarizer.fit(
    train_dataset,
    num_gpus=NUM_GPUS,
    batch_size=BATCH_SIZE,
    max_steps=MAX_STEPS,
    learning_rate_bert=LEARNING_RATE_BERT,
    learning_rate_dec=LEARNING_RATE_DEC,
    warmup_steps_bert=WARMUP_STEPS_BERT,
    warmup_steps_dec=WARMUP_STEPS_DEC,
    save_every=SAVE_EVERY,
    report_every=REPORT_EVERY * 5,
    fp16=FP16,
    # checkpoint="saved checkpoint path"
)

device is cuda


In [64]:
summarizer.save_model(MAX_STEPS, os.path.join(CACHE_PATH, "bertsumabs.pt"))

/tmp/tmp_m08v98t
saving through pytorch to /tmp/tmp_m08v98t/bertsumabs.pt


## Model Evaluation

To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](summarization_evaluation.ipynb) for setup.
For the settings in this notebook with QUICK_RUN=False, you should get ROUGE scores close to the following numbers: <br />
``
{'rouge-1': {'f': 0.34819639878321873,
             'p': 0.39977932634737307,
             'r': 0.34429079596863604},
 'rouge-2': {'f': 0.13919271352557894,
             'p': 0.16129965067780644,
             'r': 0.1372938054050938},
 'rouge-l': {'f': 0.2313282318854973,
             'p': 0.26664667422849747,
             'r': 0.22850294283399628}}
 ``
 
 Better performance can be achieved by increasing the MAX_STEPS.

In [67]:
TEST_TOP_N = 32
if not QUICK_RUN:
    TEST_TOP_N = len(test_dataset)

if NUM_GPUS:
    BATCH_SIZE = NUM_GPUS * BATCH_SIZE_PER_GPU
else:
    BATCH_SIZE = 1
    
shortened_dataset = test_dataset.shorten(top_n=TEST_TOP_N)
src = shortened_dataset.get_source()
reference_summaries = [" ".join(t).rstrip("\n") for t in shortened_dataset.get_target()]


In [70]:
generated_summaries = summarizer.predict(
    shortened_dataset, batch_size=BATCH_SIZE, num_gpus=NUM_GPUS
)
assert len(generated_summaries) == len(reference_summaries)

Generating summary:   0%|          | 0/32 [00:00<?, ?it/s]WARNING:transformers.tokenization_utils_base:Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


dataset length is 32


/home/ec2-user/SageMaker/nlp-recipes/utils_nlp/models/transformers/bertsum/predictor.py:288: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  finished_hyp = is_finished[i].nonzero().view(-1)
Generating summary: 100%|██████████| 32/32 [01:52<00:00,  3.52s/it]


In [71]:
src[1]

['( cnn ) the palestinian authority officially became the 123rd member of the international criminal court on wednesday , a step that gives the court jurisdiction over alleged crimes in palestinian territories .',
 'the formal accession was marked with a ceremony at the hague , in the netherlands , where the court is based .',
 'the palestinians signed the icc \'s founding rome statute in january , when they also accepted its jurisdiction over alleged crimes committed " in the occupied palestinian territory , including east jerusalem , since june 13 , 2014 . "',
 'later that month , the icc opened a preliminary examination into the situation in palestinian territories , paving the way for possible war crimes investigations against israelis .',
 'as members of the court , palestinians may be subject to counter-charges as well .',
 "israel and the united states , neither of which is an icc member , opposed the palestinians ' efforts to join the body .",
 'but palestinian foreign minister

In [72]:
generated_summaries[1]

'some witnesses say colorado does nothing to protect them on monday on wednesday .   one witness on security on security after being terror .  them in the u .  . s . human war "     human rights .             .'

In [73]:
reference_summaries[1]

' membership gives the icc jurisdiction over alleged crimes committed in palestinian territories since last june .    israel and the united states opposed the move , which could open the door to war crimes investigations against israelis .  '

In [83]:
rouge_scores = compute_rouge_python(cand=generated_summaries, ref=reference_summaries)
pprint.pprint(rouge_scores)

Number of candidates: 32
Number of references: 32


TypeError: __init__() got an unexpected keyword argument 'max_n'

In [80]:
?Rouge

Init signature:
Rouge(
    metrics=None,
    stats=None,
    return_lengths=False,
    raw_results=False,
    exclusive=False,
)
Docstring:      <no docstring>
File:           ~/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/rouge/rouge.py
Type:           type
Subclasses:     RougeExt


In [84]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

NameError: name 'rouge_scores' is not defined

## Prediction on a single input sample

In [85]:
source = """
But under the new rule, set to be announced in the next 48 hours, Border Patrol agents would immediately return anyone to Mexico — without any detainment and without any due process — who attempts to cross the southwestern border between the legal ports of entry. The person would not be held for any length of time in an American facility.

Although they advised that details could change before the announcement, administration officials said the measure was needed to avert what they fear could be a systemwide outbreak of the coronavirus inside detention facilities along the border. Such an outbreak could spread quickly through the immigrant population and could infect large numbers of Border Patrol agents, leaving the southwestern border defenses weakened, the officials argued.
The Trump administration plans to immediately turn back all asylum seekers and other foreigners attempting to enter the United States from Mexico illegally, saying the nation cannot risk allowing the coronavirus to spread through detention facilities and Border Patrol agents, four administration officials said.
The administration officials said the ports of entry would remain open to American citizens, green-card holders and foreigners with proper documentation. Some foreigners would be blocked, including Europeans currently subject to earlier travel restrictions imposed by the administration. The points of entry will also be open to commercial traffic."""

In [86]:
test_dataset = SummarizationDataset(
    None, source=[source], source_preprocessing=[tokenize.sent_tokenize],
)
generated_summaries = summarizer.predict(test_dataset, batch_size=1, num_gpus=NUM_GPUS)

Generating summary:   0%|          | 0/1 [00:00<?, ?it/s]WARNING:transformers.tokenization_utils_base:Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


dataset length is 1


Generating summary: 100%|██████████| 1/1 [00:03<00:00,  3.54s/it]


In [24]:
generated_summaries[0]

'empty anti - run news of himself sexually abu abusing underageageageageageageageageageage states has become comatose in the finance sector . s operations .   hispanics investigating claims , 000 customers in u . s . s assassination . s s operations s operations operations . s terrorists terrorists . s economic agent . s .           .'

## Clean up temporary folders

In [25]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_PATH):
    shutil.rmtree(CACHE_PATH, ignore_errors=True)